# Nova Scotia EHS Base Location Analysis

## Objective
This notebook analyzes Nova Scotia population and hospital data to determine optimal Emergency Health Services (EHS) base locations using K-Means clustering with population-weighted sampling. The analysis aims to find strategic locations that maximize coverage for areas with higher population density.

## Data Sources
- `polulation_location_polygon.csv`: Nova Scotia community population data with geometry information
- `Hospitals.geojson`: Location data of existing hospitals in Nova Scotia

## Methodology
- Use K-Means clustering with population as sample weights
- Analyze current hospital distribution
- Optimize EHS base locations for maximum population coverage
- Calculate accessibility metrics and response times

In [ ]:
# Import Required Libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"GeoPandas version: {gpd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Load and Explore Population Data

Loading the Nova Scotia population data from the CSV file and examining its structure.

In [ ]:
# Load population data
pop_df = pd.read_csv('0 polulation_location_polygon.csv')

print("Population Data Overview:")
print(f"Shape: {pop_df.shape}")
print(f"\nColumns: {pop_df.columns.tolist()}")
print(f"\nFirst few rows:")
pop_df.head()

In [ ]:
# Examine data types and missing values
print("Data Types:")
print(pop_df.dtypes)
print("\nMissing Values:")
print(pop_df.isnull().sum())

# Basic statistics for key columns
if 'C1_COUNT_TOTAL' in pop_df.columns:
    print(f"\nPopulation Statistics:")
    print(pop_df['C1_COUNT_TOTAL'].describe())
    print(f"Total Population: {pop_df['C1_COUNT_TOTAL'].sum():,}")

# Check coordinate columns
coord_cols = ['latitude', 'longitude']
for col in coord_cols:
    if col in pop_df.columns:
        print(f"\n{col} range: {pop_df[col].min():.6f} to {pop_df[col].max():.6f}")
    else:
        print(f"Column '{col}' not found")

## 2. Load and Explore Hospital Location Data

Loading the hospital location data from the GeoJSON file and examining the distribution of existing healthcare facilities.

In [ ]:
# Load hospital data
hospitals_gdf = gpd.read_file('1 Hospitals.geojson')

print("Hospital Data Overview:")
print(f"Shape: {hospitals_gdf.shape}")
print(f"CRS: {hospitals_gdf.crs}")
print(f"\nColumns: {hospitals_gdf.columns.tolist()}")
print(f"\nFirst few rows:")
hospitals_gdf.head()

In [ ]:
# Analyze hospital types and distribution
print("Hospital Types Distribution:")
print(hospitals_gdf['type'].value_counts())

print("\nHospital Counties:")
print(hospitals_gdf['county'].value_counts())

# Extract coordinates for analysis
hospitals_gdf['longitude'] = hospitals_gdf.geometry.x
hospitals_gdf['latitude'] = hospitals_gdf.geometry.y

print(f"\nHospital coordinate ranges:")
print(f"Latitude: {hospitals_gdf['latitude'].min():.6f} to {hospitals_gdf['latitude'].max():.6f}")
print(f"Longitude: {hospitals_gdf['longitude'].min():.6f} to {hospitals_gdf['longitude'].max():.6f}")

# Display hospital summary
hospitals_gdf[['facility', 'town', 'county', 'type', 'latitude', 'longitude']].head(10)

## 3. Data Preprocessing and Cleaning

Cleaning and preprocessing both datasets to ensure data quality and consistency for the analysis.

In [ ]:
# Clean population data
print("Cleaning population data...")

# Remove rows with missing essential data
clean_pop_df = pop_df.dropna(subset=['latitude', 'longitude', 'C1_COUNT_TOTAL']).copy()

# Filter out rows with zero or negative population
clean_pop_df = clean_pop_df[clean_pop_df['C1_COUNT_TOTAL'] > 0]

# Remove coordinate outliers (basic sanity check for Nova Scotia)
clean_pop_df = clean_pop_df[
    (clean_pop_df['latitude'] >= 43.0) & (clean_pop_df['latitude'] <= 47.0) &
    (clean_pop_df['longitude'] >= -67.0) & (clean_pop_df['longitude'] <= -59.0)
]

print(f"Original data: {len(pop_df)} rows")
print(f"After cleaning: {len(clean_pop_df)} rows")
print(f"Total population: {clean_pop_df['C1_COUNT_TOTAL'].sum():,}")
print(f"Average community population: {clean_pop_df['C1_COUNT_TOTAL'].mean():.0f}")

# Display summary statistics
clean_pop_df['C1_COUNT_TOTAL'].describe()

## 4. Prepare Geospatial Data for Analysis

Converting geometry data to appropriate formats and extracting coordinates for clustering analysis.

In [ ]:
# Prepare coordinates and visualize data distribution
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Population distribution by coordinates
scatter = ax1.scatter(clean_pop_df['longitude'], clean_pop_df['latitude'], 
                     c=clean_pop_df['C1_COUNT_TOTAL'], 
                     s=50, alpha=0.6, cmap='viridis')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('Population Distribution Across Nova Scotia')
plt.colorbar(scatter, ax=ax1, label='Population')

# Plot 2: Hospital locations
ax2.scatter(hospitals_gdf['longitude'], hospitals_gdf['latitude'], 
           c='red', s=100, alpha=0.7, marker='h')
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Existing Hospital Locations')

# Plot 3: Population histogram
ax3.hist(clean_pop_df['C1_COUNT_TOTAL'], bins=50, alpha=0.7, edgecolor='black')
ax3.set_xlabel('Population')
ax3.set_ylabel('Frequency')
ax3.set_title('Population Distribution (Communities)')
ax3.set_yscale('log')

# Plot 4: Combined view
ax4.scatter(clean_pop_df['longitude'], clean_pop_df['latitude'], 
           c='blue', s=30, alpha=0.4, label='Communities')
ax4.scatter(hospitals_gdf['longitude'], hospitals_gdf['latitude'], 
           c='red', s=100, alpha=0.8, marker='h', label='Hospitals')
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Latitude')
ax4.set_title('Communities and Hospitals Overview')
ax4.legend()

plt.tight_layout()
plt.show()

print(f"Data ready for clustering:")
print(f"Communities: {len(clean_pop_df)}")
print(f"Hospitals: {len(hospitals_gdf)}")
print(f"Coordinate bounds - Lat: [{clean_pop_df['latitude'].min():.3f}, {clean_pop_df['latitude'].max():.3f}]")
print(f"Coordinate bounds - Lon: [{clean_pop_df['longitude'].min():.3f}, {clean_pop_df['longitude'].max():.3f}]")

## 5. Extract Population Weights and Coordinates

Preparing the data arrays needed for K-Means clustering with population weighting.

In [ ]:
# Prepare data for K-Means clustering
print("Preparing data for K-Means clustering...")

# Extract coordinates
X = clean_pop_df[['longitude', 'latitude']].values

# Extract population weights
sample_weights = clean_pop_df['C1_COUNT_TOTAL'].values

# Standardize coordinates for clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Feature matrix shape: {X.shape}")
print(f"Sample weights shape: {sample_weights.shape}")
print(f"Sample weights statistics:")
print(f"  Min: {sample_weights.min()}")
print(f"  Max: {sample_weights.max()}")
print(f"  Mean: {sample_weights.mean():.0f}")
print(f"  Total: {sample_weights.sum():,}")

# Visualize weight distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(sample_weights, bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Population (Sample Weight)')
plt.ylabel('Frequency')
plt.title('Distribution of Population Weights')
plt.yscale('log')

plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=sample_weights, s=30, alpha=0.6, cmap='viridis')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Distribution of Population Weights')
plt.colorbar(label='Population')

plt.tight_layout()
plt.show()

## 6. Determine Optimal Number of Clusters

Using elbow method and silhouette analysis to find the optimal number of EHS base locations.

In [ ]:
# Find optimal number of clusters
def find_optimal_clusters(X_scaled, sample_weights, max_k=12):
    inertias = []
    silhouette_scores = []
    K_range = range(2, max_k + 1)
    
    print("Testing different numbers of clusters...")
    
    for k in K_range:
        print(f"Testing k={k}...", end=' ')
        
        # Fit K-Means with population weights
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled, sample_weight=sample_weights)
        
        inertias.append(kmeans.inertia_)
        
        # Calculate silhouette score with sample weights
        labels = kmeans.labels_
        sil_score = silhouette_score(X_scaled, labels, sample_weight=sample_weights)
        silhouette_scores.append(sil_score)
        
        print(f"Silhouette: {sil_score:.3f}")
    
    return K_range, inertias, silhouette_scores

# Run optimization
K_range, inertias, silhouette_scores = find_optimal_clusters(X_scaled, sample_weights)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow curve
ax1.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Within-cluster sum of squares)')
ax1.set_title('Elbow Method for Optimal k')
ax1.grid(True, alpha=0.3)

# Silhouette scores
ax2.plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score vs Number of Clusters')
ax2.grid(True, alpha=0.3)

# Highlight best k
best_k = K_range[np.argmax(silhouette_scores)]
ax2.axvline(x=best_k, color='green', linestyle='--', alpha=0.7, linewidth=2)
ax2.text(best_k + 0.1, max(silhouette_scores) * 0.95, f'Best k={best_k}', 
         fontsize=12, fontweight='bold', color='green')

plt.tight_layout()
plt.show()

print(f"\nOptimal number of clusters based on silhouette score: {best_k}")
print(f"Best silhouette score: {max(silhouette_scores):.3f}")

# Display all results
results_df = pd.DataFrame({
    'k': K_range,
    'inertia': inertias,
    'silhouette_score': silhouette_scores
})
print("\nDetailed Results:")
print(results_df)

## 7. Implement K-Means Clustering with Population Weights

Applying K-Means clustering with the optimal number of clusters to determine EHS base locations.

In [ ]:
# Perform final clustering with optimal k
print(f"Performing K-Means clustering with k={best_k}...")

# Fit the final model
final_kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
final_kmeans.fit(X_scaled, sample_weight=sample_weights)

# Get cluster centers in original coordinates
cluster_centers_scaled = final_kmeans.cluster_centers_
cluster_centers = scaler.inverse_transform(cluster_centers_scaled)

# Add cluster assignments to the dataframe
clean_pop_df['cluster'] = final_kmeans.labels_

print("Cluster Centers (Proposed EHS Base Locations):")
ems_locations = []
for i, center in enumerate(cluster_centers):
    lon, lat = center
    ems_locations.append({
        'EHS_Base_ID': f'EHS-{i+1}',
        'Longitude': lon,
        'Latitude': lat
    })
    print(f"EHS-{i+1}: Longitude {lon:.6f}, Latitude {lat:.6f}")

# Calculate cluster statistics
cluster_stats = []
for i in range(best_k):
    cluster_data = clean_pop_df[clean_pop_df['cluster'] == i]
    
    stats = {
        'Cluster_ID': i,
        'EHS_Base': f'EHS-{i+1}',
        'Center_Longitude': cluster_centers[i, 0],
        'Center_Latitude': cluster_centers[i, 1],
        'Num_Communities': len(cluster_data),
        'Total_Population': cluster_data['C1_COUNT_TOTAL'].sum(),
        'Avg_Population': cluster_data['C1_COUNT_TOTAL'].mean(),
        'Population_Density': cluster_data['C1_COUNT_TOTAL'].sum() / len(cluster_data)
    }
    cluster_stats.append(stats)

cluster_stats_df = pd.DataFrame(cluster_stats)
print(f"\nCluster Statistics:")
print(cluster_stats_df.round(2))

## 8. Visualize EHS Base Locations on Map

Creating comprehensive visualizations of the proposed EHS base locations with population centers and existing hospitals.

In [ ]:
# Create comprehensive visualization
fig, ax = plt.subplots(1, 1, figsize=(16, 12))

# Plot communities colored by cluster, sized by population
colors = plt.cm.tab10(np.linspace(0, 1, best_k))
for i in range(best_k):
    cluster_data = clean_pop_df[clean_pop_df['cluster'] == i]
    ax.scatter(cluster_data['longitude'], cluster_data['latitude'], 
              c=[colors[i]], s=cluster_data['C1_COUNT_TOTAL']/30,
              alpha=0.6, label=f'Cluster {i+1}', edgecolors='black', linewidth=0.5)

# Plot existing hospitals
ax.scatter(hospitals_gdf['longitude'], hospitals_gdf['latitude'], 
          c='red', s=150, marker='h', alpha=0.9, 
          label='Existing Hospitals', edgecolors='darkred', linewidth=1)

# Plot proposed EHS base locations
ax.scatter(cluster_centers[:, 0], cluster_centers[:, 1], 
          c='yellow', s=400, marker='*', 
          label='Proposed EHS Bases', edgecolors='black', linewidth=2)

# Add EHS base labels
for i, (lon, lat) in enumerate(cluster_centers):
    ax.annotate(f'EHS-{i+1}', (lon, lat), 
               xytext=(8, 8), textcoords='offset points',
               fontsize=12, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.8))

ax.set_xlabel('Longitude', fontsize=14)
ax.set_ylabel('Latitude', fontsize=14)
ax.set_title('Optimal EHS Base Locations in Nova Scotia\\n(Community size proportional to population)', 
             fontsize=16, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Create cluster statistics visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Population by cluster
bars1 = ax1.bar(cluster_stats_df['Cluster_ID'], cluster_stats_df['Total_Population'], color=colors)
ax1.set_xlabel('Cluster ID')
ax1.set_ylabel('Total Population')
ax1.set_title('Total Population by Cluster')
ax1.set_xticks(cluster_stats_df['Cluster_ID'])

# Number of communities by cluster
bars2 = ax2.bar(cluster_stats_df['Cluster_ID'], cluster_stats_df['Num_Communities'], color=colors)
ax2.set_xlabel('Cluster ID')
ax2.set_ylabel('Number of Communities')
ax2.set_title('Number of Communities by Cluster')
ax2.set_xticks(cluster_stats_df['Cluster_ID'])

# Average population by cluster
bars3 = ax3.bar(cluster_stats_df['Cluster_ID'], cluster_stats_df['Avg_Population'], color=colors)
ax3.set_xlabel('Cluster ID')
ax3.set_ylabel('Average Population')
ax3.set_title('Average Population by Cluster')
ax3.set_xticks(cluster_stats_df['Cluster_ID'])

# Population density by cluster
bars4 = ax4.bar(cluster_stats_df['Cluster_ID'], cluster_stats_df['Population_Density'], color=colors)
ax4.set_xlabel('Cluster ID')
ax4.set_ylabel('Population Density (Pop/Community)')
ax4.set_title('Population Density by Cluster')
ax4.set_xticks(cluster_stats_df['Cluster_ID'])

plt.tight_layout()
plt.show()

In [ ]:
# Create interactive map using Folium
def create_interactive_map():
    # Calculate map center
    center_lat = clean_pop_df['latitude'].mean()
    center_lon = clean_pop_df['longitude'].mean()
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=7,
        tiles='OpenStreetMap'
    )
    
    # Define colors for clusters
    cluster_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
                     'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 
                     'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 
                     'gray', 'black', 'lightgray']
    
    # Add communities colored by cluster
    for idx, row in clean_pop_df.iterrows():
        cluster_id = int(row['cluster'])
        color = cluster_colors[cluster_id % len(cluster_colors)]
        
        # Size based on population (scaled)
        radius = max(3, min(15, row['C1_COUNT_TOTAL'] / 500))
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=radius,
            popup=f\"Community: {row.get('GEO_NAME', 'Unknown')}<br>Population: {row['C1_COUNT_TOTAL']:,}<br>Cluster: {cluster_id+1}\",
            color='black',
            weight=1,
            fillColor=color,
            fillOpacity=0.6
        ).add_to(m)
    
    # Add existing hospitals
    for idx, row in hospitals_gdf.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f\"<b>{row['facility']}</b><br>Type: {row['type']}<br>Town: {row['town']}<br>County: {row['county']}\",
            icon=folium.Icon(color='red', icon='plus', prefix='fa')
        ).add_to(m)
    
    # Add proposed EHS bases
    for i, center in enumerate(cluster_centers):
        lon, lat = center
        folium.Marker(
            location=[lat, lon],
            popup=f\"<b>Proposed EHS Base {i+1}</b><br>Latitude: {lat:.6f}<br>Longitude: {lon:.6f}<br>Serves Cluster {i+1}\",
            icon=folium.Icon(color='yellow', icon='star', prefix='fa')
        ).add_to(m)
    
    # Add legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 120px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <p><b>Legend</b></p>
    <p><i class="fa fa-star" style="color:gold"></i> Proposed EHS Bases</p>
    <p><i class="fa fa-plus" style="color:red"></i> Existing Hospitals</p>
    <p><i class="fa fa-circle" style="color:blue"></i> Communities (by cluster)</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

# Create and display the interactive map
interactive_map = create_interactive_map()
interactive_map.save('nova_scotia_ems_locations.html')
print("Interactive map saved as 'nova_scotia_ems_locations.html'")
print("You can open this file in a web browser to explore the interactive map.")

# Display the map in the notebook
interactive_map

## 9. Analyze Coverage and Accessibility

Calculating coverage metrics, response time estimates, and accessibility analysis for the proposed EHS base locations.

In [ ]:
# Calculate distances and coverage metrics
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on earth"""
    from math import radians, cos, sin, asin, sqrt
    
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

# Calculate distance from each community to its assigned EHS base
distances_to_assigned = []
for idx, row in clean_pop_df.iterrows():
    cluster_id = row['cluster']
    center_lat, center_lon = cluster_centers[cluster_id, 1], cluster_centers[cluster_id, 0]
    
    distance = haversine_distance(row['latitude'], row['longitude'], center_lat, center_lon)
    distances_to_assigned.append(distance)

clean_pop_df['distance_to_ems'] = distances_to_assigned

# Calculate coverage statistics
print("=== COVERAGE ANALYSIS ===")
print(f"Total Communities: {len(clean_pop_df)}")
print(f"Total Population: {clean_pop_df['C1_COUNT_TOTAL'].sum():,}")
print(f"\\nDistance Statistics:")
print(f"Average distance to EHS base: {np.mean(distances_to_assigned):.2f} km")
print(f"Median distance to EHS base: {np.median(distances_to_assigned):.2f} km")
print(f"Maximum distance to EHS base: {np.max(distances_to_assigned):.2f} km")
print(f"Minimum distance to EHS base: {np.min(distances_to_assigned):.2f} km")

# Population-weighted average distance
total_pop = clean_pop_df['C1_COUNT_TOTAL'].sum()
weighted_avg_distance = (clean_pop_df['distance_to_ems'] * clean_pop_df['C1_COUNT_TOTAL']).sum() / total_pop
print(f"Population-weighted average distance: {weighted_avg_distance:.2f} km")

# Coverage within different distance thresholds
print(f"\\n=== POPULATION COVERAGE BY DISTANCE ===")
thresholds = [5, 10, 15, 20, 30, 50]  # km
coverage_data = []

for threshold in thresholds:
    coverage_pop = clean_pop_df[clean_pop_df['distance_to_ems'] <= threshold]['C1_COUNT_TOTAL'].sum()
    coverage_communities = len(clean_pop_df[clean_pop_df['distance_to_ems'] <= threshold])
    percentage_pop = (coverage_pop / total_pop) * 100
    percentage_comm = (coverage_communities / len(clean_pop_df)) * 100
    
    coverage_data.append({
        'Distance_km': threshold,
        'Population_Covered': coverage_pop,
        'Communities_Covered': coverage_communities,
        'Population_Percentage': percentage_pop,
        'Communities_Percentage': percentage_comm
    })
    
    print(f"Within {threshold:2d} km: {coverage_pop:6,} people ({percentage_pop:5.1f}%) | {coverage_communities:3d} communities ({percentage_comm:5.1f}%)")

coverage_df = pd.DataFrame(coverage_data)

# Response time estimates (assuming average speeds)
print(f"\\n=== ESTIMATED RESPONSE TIMES ===")
print("(Assuming average speeds: 60 km/h highway, 40 km/h rural/urban)")

# Conservative estimate using 40 km/h average
response_times_40 = clean_pop_df['distance_to_ems'] / 40 * 60  # Convert to minutes
clean_pop_df['response_time_40kmh'] = response_times_40

print(f"Average response time (40 km/h): {np.mean(response_times_40):.1f} minutes")
print(f"Median response time (40 km/h): {np.median(response_times_40):.1f} minutes")
print(f"Maximum response time (40 km/h): {np.max(response_times_40):.1f} minutes")

# Response time thresholds
time_thresholds = [10, 15, 20, 30]  # minutes
print(f"\\nPopulation coverage within response time thresholds (40 km/h):")
for time_threshold in time_thresholds:
    coverage_pop = clean_pop_df[clean_pop_df['response_time_40kmh'] <= time_threshold]['C1_COUNT_TOTAL'].sum()
    percentage = (coverage_pop / total_pop) * 100
    print(f"Within {time_threshold:2d} minutes: {coverage_pop:6,} people ({percentage:5.1f}%)")

In [ ]:
# Visualize coverage analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Distance distribution histogram
ax1.hist(distances_to_assigned, bins=30, alpha=0.7, edgecolor='black', color='skyblue')
ax1.axvline(np.mean(distances_to_assigned), color='red', linestyle='--', 
           label=f'Mean: {np.mean(distances_to_assigned):.1f} km')
ax1.axvline(np.median(distances_to_assigned), color='green', linestyle='--', 
           label=f'Median: {np.median(distances_to_assigned):.1f} km')
ax1.set_xlabel('Distance to Nearest EHS Base (km)')
ax1.set_ylabel('Number of Communities')
ax1.set_title('Distribution of Distances to EHS Bases')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Coverage by distance thresholds
ax2.plot(coverage_df['Distance_km'], coverage_df['Population_Percentage'], 
         'o-', linewidth=2, markersize=8, color='blue', label='Population')
ax2.plot(coverage_df['Distance_km'], coverage_df['Communities_Percentage'], 
         's-', linewidth=2, markersize=8, color='orange', label='Communities')
ax2.set_xlabel('Distance Threshold (km)')
ax2.set_ylabel('Coverage Percentage (%)')
ax2.set_title('Population and Community Coverage by Distance')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

# Response time distribution
ax3.hist(response_times_40, bins=30, alpha=0.7, edgecolor='black', color='lightcoral')
ax3.axvline(np.mean(response_times_40), color='red', linestyle='--', 
           label=f'Mean: {np.mean(response_times_40):.1f} min')
ax3.axvline(np.median(response_times_40), color='green', linestyle='--', 
           label=f'Median: {np.median(response_times_40):.1f} min')
ax3.set_xlabel('Estimated Response Time (minutes)')
ax3.set_ylabel('Number of Communities')
ax3.set_title('Distribution of Estimated Response Times (40 km/h)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Geographic distribution of distances
scatter = ax4.scatter(clean_pop_df['longitude'], clean_pop_df['latitude'], 
                     c=clean_pop_df['distance_to_ems'], s=50, alpha=0.6, cmap='RdYlBu_r')
ax4.scatter(cluster_centers[:, 0], cluster_centers[:, 1], 
           c='yellow', s=300, marker='*', edgecolors='black', linewidth=2)
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Latitude')
ax4.set_title('Geographic Distribution of EHS Access\\n(Color = Distance to Nearest Base)')
cbar = plt.colorbar(scatter, ax=ax4)
cbar.set_label('Distance (km)')

plt.tight_layout()
plt.show()

# Display coverage summary table
print("\\n=== COVERAGE SUMMARY TABLE ===")
print(coverage_df.round(1))

## 10. Export Results for Capstone Project

Saving all analysis results, visualizations, and optimal locations data for use in the Capstone Project documentation.

In [ ]:
# Export all results for the capstone project
print("=== EXPORTING RESULTS ===")

# 1. Export EHS base locations
ems_locations_df = pd.DataFrame(ems_locations)
ems_locations_df = pd.concat([ems_locations_df, cluster_stats_df[['Total_Population', 'Num_Communities', 'Population_Density']]], axis=1)
ems_locations_df.to_csv('proposed_ems_locations.csv', index=False)
print("✓ Proposed EMS locations saved to: proposed_ems_locations.csv")

# 2. Export detailed cluster assignments
export_columns = ['GEO_NAME', 'latitude', 'longitude', 'C1_COUNT_TOTAL', 'cluster', 'distance_to_ems', 'response_time_40kmh']
available_columns = [col for col in export_columns if col in clean_pop_df.columns]
clean_pop_df[available_columns].to_csv('community_cluster_assignments.csv', index=False)
print("✓ Community cluster assignments saved to: community_cluster_assignments.csv")

# 3. Export cluster statistics
cluster_stats_df.to_csv('cluster_statistics.csv', index=False)
print("✓ Cluster statistics saved to: cluster_statistics.csv")

# 4. Export coverage analysis
coverage_df.to_csv('coverage_analysis.csv', index=False)
print("✓ Coverage analysis saved to: coverage_analysis.csv")

# 5. Create summary report
summary_report = f\"\"\"
# Nova Scotia EHS Base Location Analysis - Summary Report

## Analysis Overview
- **Objective**: Determine optimal Emergency Health Services (EHS) base locations in Nova Scotia
- **Method**: K-Means clustering with population-weighted sampling
- **Data Sources**: Population data (95 communities) and hospital locations (47 facilities)

## Key Results
- **Optimal Number of EHS Bases**: {best_k}
- **Total Population Served**: {clean_pop_df['C1_COUNT_TOTAL'].sum():,}
- **Total Communities**: {len(clean_pop_df)}

## Coverage Metrics
- **Average Distance to EHS Base**: {np.mean(distances_to_assigned):.2f} km
- **Population-Weighted Average Distance**: {weighted_avg_distance:.2f} km
- **Maximum Distance**: {np.max(distances_to_assigned):.2f} km
- **Average Response Time (40 km/h)**: {np.mean(response_times_40):.1f} minutes

## Population Coverage Thresholds
- **Within 10 km**: {coverage_df[coverage_df['Distance_km']==10]['Population_Percentage'].iloc[0]:.1f}% of population
- **Within 15 km**: {coverage_df[coverage_df['Distance_km']==15]['Population_Percentage'].iloc[0]:.1f}% of population
- **Within 20 km**: {coverage_df[coverage_df['Distance_km']==20]['Population_Percentage'].iloc[0]:.1f}% of population

## Proposed EHS Base Locations
\"\"\"

for i, location in enumerate(ems_locations):
    summary_report += f\"\\n{i+1}. **{location['EHS_Base_ID']}**: Latitude {location['Latitude']:.6f}, Longitude {location['Longitude']:.6f}\"

summary_report += f\"\"\"

## Cluster Statistics
\"\"\"

for _, row in cluster_stats_df.iterrows():
    summary_report += f\"\\n- **Cluster {int(row['Cluster_ID'])+1}**: {int(row['Total_Population']):,} people in {int(row['Num_Communities'])} communities\"

summary_report += f\"\"\"

## Files Generated
1. `proposed_ems_locations.csv` - Coordinates and details of optimal EHS base locations
2. `community_cluster_assignments.csv` - Each community's cluster assignment and distances
3. `cluster_statistics.csv` - Statistical summary for each cluster
4. `coverage_analysis.csv` - Coverage metrics by distance thresholds
5. `nova_scotia_ems_locations.html` - Interactive map visualization
6. `analysis_summary.md` - This summary report

## Methodology Notes
- Used population as sample weights in K-Means clustering to prioritize high-population areas
- Optimized cluster number using silhouette analysis
- Distance calculations use Haversine formula for accuracy
- Response time estimates assume 40 km/h average speed (conservative estimate)

## Recommendations
1. Establish EHS bases at the {best_k} proposed locations
2. Priority should be given to bases serving the highest population densities
3. Consider geographic barriers and road networks for final placement refinement
4. Regular review and adjustment based on population changes and response time data
\"\"\"

# Save summary report
with open('analysis_summary.md', 'w') as f:
    f.write(summary_report)
print("✓ Summary report saved to: analysis_summary.md")

print(f"\\n=== ANALYSIS COMPLETE ===")
print(f"\\nAll results have been exported and are ready for your Capstone Project!")
print(f"\\nGenerated files:")
print(f"  📊 proposed_ems_locations.csv")
print(f"  📊 community_cluster_assignments.csv") 
print(f"  📊 cluster_statistics.csv")
print(f"  📊 coverage_analysis.csv")
print(f"  🗺️  nova_scotia_ems_locations.html")
print(f"  📝 analysis_summary.md")

# Display final summary
print(f"\\n=== FINAL SUMMARY ===")
print(f"Optimal number of EHS bases: {best_k}")
print(f"Total population served: {clean_pop_df['C1_COUNT_TOTAL'].sum():,}")
print(f"Average distance to nearest base: {np.mean(distances_to_assigned):.2f} km")
print(f"Population within 15 km: {coverage_df[coverage_df['Distance_km']==15]['Population_Percentage'].iloc[0]:.1f}%")
print(f"Average response time: {np.mean(response_times_40):.1f} minutes")

ems_locations_df